# 2장 시계열 데이터의 발견 및 다루기

In [1]:
import pandas as pd

YearJoined = pd.read_csv('Ch02/data/year_joined.csv')

YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


In [22]:
emails = pd.read_csv('Ch02/data/emails.csv')

emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [23]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


In [24]:
(max(emails[emails.user==998].week) - min(emails[emails.user==998].week)).days/7

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [25]:
max(emails[emails.user==998].week)

'2018-05-28 00:00:00'

In [26]:
import datetime
datetime.datetime.strptime(max(emails[emails.user==998].week), '%Y-%m-%d %H:%M:%S')

datetime.datetime(2018, 5, 28, 0, 0)

In [27]:
(datetime.datetime.strptime(max(emails[emails.user==998].week), '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(min(emails[emails.user==998].week), '%Y-%m-%d %H:%M:%S')).days/7

25.0

In [28]:
emails[emails.user==998].shape

(24, 3)

In [31]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [32]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value = 0).reset_index()

all_email.columns=['week', 'user', 'emailsOpened']